In [2]:
import os.path
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd

from utils.constants import Events, datetimes
from utils.plots import plot

In [3]:
EVENT: Events = "Forbush Decrease"
event_replace = EVENT.replace(" ", "")
series_figure_path = "./figures/{event}/{date}/series"

## Plot both (Original and EWM) separately

In [25]:
paths = {
    date: list(Path(f"./data/ForbushDecrease/{date}").glob("*.csv"))
    for date in datetimes
}

In [29]:
for date, path in paths.items():
    for file in path:
        # Extract metadata from filename
        splits = file.stem.split("-")
        station_name = splits[0].split("_", 1)[0].upper()
        window_size = int(splits[1].split("_", 1)[1])
        ewm_alpha = splits[2].split("_")[2] if len(splits) == 3 else None
        bounds = pd.to_datetime(datetimes[date]["stations"][station_name])

        # Load data
        df = pd.read_csv(file, parse_dates=["datetime"])

        # Plot data
        fig, ax = plt.subplots(figsize=(14, 6))

        # Time Series data
        plot(df, ax=ax, freq_hours=2, are_metrics=False)

        # Vertical lines for event bounds
        ax.axvline(bounds[0], color="black", linestyle="--")
        ax.axvline(bounds[1], color="black", linestyle="--")

        # Title and save path
        if ewm_alpha:
            title = (
                rf"{station_name} Station | {EVENT} | EWM $\alpha={ewm_alpha}$"
            )
            ewm_alpha_suffix = f"ewm_alpha_{ewm_alpha}"
        else:
            title = f"{station_name} Station | {EVENT}"
            ewm_alpha_suffix = ""

        ax.set(title=title, ylabel="Count")
        save_path = series_figure_path.format(event=event_replace, date=date)
        if not os.path.exists(save_path):
            os.makedirs(save_path, exist_ok=True)

        plt.savefig(
            f"{save_path}/{station_name}{f'-{ewm_alpha_suffix}' if ewm_alpha_suffix else ''}.pdf"
        )
        plt.close()

## Plot both in one (EWM $\alpha$ could be different just for plots)

In [31]:
paths_original = {
    date: list(
        Path(f"./data/ForbushDecrease/{date}").glob("*-windowsize_130.csv")
    )
    for date in datetimes
}

ewm_alpha = 0.15

In [32]:
for date, path in paths_original.items():
    for file in path:
        # Extract metadata from filename
        splits = file.stem.split("-")
        station_name = splits[0].split("_", 1)[0].upper()
        window_size = int(splits[1].split("_", 1)[1])
        bounds = pd.to_datetime(datetimes[date]["stations"][station_name])

        # Load data
        df = pd.read_csv(file, parse_dates=["datetime"])

        # Plot data
        fig, ax = plt.subplots(figsize=(14, 6))

        # Time Series data
        plot(
            df,
            ax=ax,
            freq_hours=2,
            are_metrics=False,
            kwargs_metrics_false={"label": "original"},
        )

        # EWM data
        df_ewm = df.copy()
        df_ewm["value"] = df_ewm["value"].ewm(alpha=ewm_alpha).mean()
        plot(
            df_ewm,
            ax=ax,
            are_metrics=False,
            kwargs_metrics_false={
                "label": "EWM",
                "color": "tab:orange",
            },
        )

        # Vertical lines for event bounds
        ax.axvline(bounds[0], color="black", linestyle="--")
        ax.axvline(bounds[1], color="black", linestyle="--")

        title = rf"{station_name} Station | {EVENT} | EWM $\alpha={ewm_alpha}$"
        ax.set(title=title, ylabel="Count")

        save_path = series_figure_path.format(event=event_replace, date=date)
        if not os.path.exists(save_path):
            os.makedirs(save_path, exist_ok=True)

        plt.savefig(f"{save_path}/{station_name}-both.pdf")
        plt.close()